# Использование HMM на реальных данных

Данный блокнот можно немного преувеличив минимальным прототипом аватара. Он демонстрирует выделение из видео признаков типа [F.A.C.S. Action Units]((https://en.wikipedia.org/wiki/Facial_Action_Coding_System#Method), обучение по ним скрытой марковской цепи (Hidden Markov Model, HMM) и анимацию предсказания.

При анализе данных используется цепочка,
```
Image -> Land Marks -> FACS Action Units -> HMM
```
а в обратном ходе, при демонстрации аватаром лицевой анимации практически идентичная обратная
```
HMM -> FACS Action Units -> Animator -> Image
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import scipy
import itertools
from feat.data import Fex
from feat.utils.io import read_feat

from hmmlearn import hmm
import tqdm
import tqdm.contrib.itertools

import graphviz

## Выделение признаков

Данные были получены при помощи библиотеки `py-feat`, которая умеет выделять из видео не только эмоции и опорные точки лица, но и Action Units системы FACS, что позволяет достаточно просто обратить преобразование и анимировать изображение.

In [ ]:
from feat import Detector
from feat.utils.io import get_test_data_path
import os

enabled     = False         # False means do nothing
video_path  = '/fix/me'
csv_path    = '/fix/me/too'
fps         = 25
skip_frames = fps           # start detection every second
face_detection_threshold = 0.95

if enabled:
    detector = Detector()
    
    video_prediction = detector.detect_video(
        video_path,
        skip_frames=skip_frames,
        face_detection_threshold=face_detection_threshold
    )
    
    print(f"Saving to csv {csv_path}")
    video_prediction.to_csv(csv_path)


Класс `feat.data.Fex`, экземпляром которого является video_prediction является оберткой над pandas DataFrame. Данные из него можно сохранять стандартными методами `pandas`, прочитать данные из csv обратно в `feat.data.Fex` можно при помощи `feat.utils.io.read_feat`.

In [ ]:
data_path = '../data/2019-11-05_17-34-46-Cam0.csv'

data = read_feat(data_path)
display(data)

In [ ]:
axes = data.emotions.plot()
observation = data.aus
labels = list(data.aus.columns)

## Обучение HMM на FACS Action Units

In [ ]:
train_validate_fraction = .75   # train/validate
min_state_n             = 2     # must be positive
max_state_n             = 20    # must be grater than min_state_n
attemptes_to_fit_n      = 20    # number of attempts to fit a model
                                # initial random state will different for each attempt
show_progress_bar       = True  # show progress bar of overall training process
print_log               = False # print a message per a trained model
verbose                 = False # print verbose log of training for every model
random_state            = 0     # random state seed, must be unsigned 32 bit integer
n_samples_after         = 200   # number of states to generate after validating sequence
raw_data_multiplier     = 100   # number to multiply data before conservation to integer 

X = np.array((raw_data_multiplier * (observation - np.min(observation))).astype(np.int_)) # for Poisson HMM all values must be positive

train_validate_border = int(X.shape[0] * train_validate_fraction)

X_train    = X[:train_validate_border]
X_validate = X[train_validate_border:]

scores = list()
models = list()

product = tqdm.contrib.itertools.product if show_progress_bar else itertools.product

try:
    for idx, n_components in product(range(attemptes_to_fit_n),
                                     range(min_state_n, max_state_n)):
        model = hmm.PoissonHMM(n_components = n_components,
                               random_state=idx,
                               verbose=verbose)
        
        model.fit(X_train)
        models.append(model)
        scores.append(model.score(X_validate))
        if print_log:
            print(f'Converged: {model.monitor_.converged}'
                  f'\tScore: {scores[-1]}')
except:
    raise ValueError("PoissonHMM firring failed. Check if all values"
                     "are positve integers and tere are enough values"
                     "to fit hmm with souch amount of states.")

# get the best model
model = models[np.argmax(scores)]
print(f'The best model had a score of {max(scores)} and '
      f'{model.n_components} components')

# use the Viterbi algorithm to predict the most likely sequence of states
# of the given the model for the train sequence
states_train       = model.predict(X_train)
# generate states with the model without any information from real data
_, states_validate = model.sample(n_samples = len(X_validate) + n_samples_after,
                              random_state = random_state,
                              currstate = states_train[-1])
states = np.concatenate([states_train, states_validate])

Демонстрации реальных и сгенерированных данных на одном графике

In [ ]:
# plot model states over time
X_label         = list(map(lambda label: f"real {label}", labels))
fited_label     = list(map(lambda label: f"predicted {label}", labels))
first_au_num    = 16
last_au_num     = 100 # feel free to set value greater then the maximum AU number
aus_range       = range(first_au_num, min(X.shape[1], model.lambdas_[states].shape[1], last_au_num))

fig, ax = plt.subplots()
for i in aus_range:
    line, = ax.plot(X[:,i],
            linestyle="--",
            label = X_label[i])
    ax.plot(model.lambdas_[states][:,i],
            color = line.get_color(),
            label = fited_label[i])
ax.axvline(x = train_validate_border,
           linestyle = 'dotted',
           color = 'gray',
           label = 'tarain/validate border')
ax.axvline(x = X.shape[0],
           linestyle = 'dashed',
           color = 'gray',
           label = 'end of real data')
ax.legend(loc = (1.05,0))
ax.set_title('Predicted data and real data comparation')
ax.set_xlabel('Frame number')
fig.show()

## Визуализация внутреннего устройства HMM
Внутренние состояния HMM организованы в граф, на ребрах которого отмечены вероятности переходов. Изобразим его матрицу смежности и сам граф.

In [ ]:
fig, ax = plt.subplots()
# ax.imshow(model.transmat_, aspect='auto', cmap='spring')
plot = ax.pcolormesh(model.transmat_,
               edgecolors='white',
               linewidth=1)
fig.colorbar(plot)
ax.invert_yaxis()
ax.set_aspect('equal')
ax.set_title('Transition Matrix')
ax.set_xlabel('State To')
ax.set_ylabel('State From')
fig.show()

In [ ]:
min_prob_to_plot_an_edge = 0.01
float_format = "{0:0.2f}"

matrix = np.array(model.transmat_)

assert(len(matrix.shape) == 2)
assert(matrix.shape[0] == matrix.shape[0])

dot = graphviz.Digraph('HMM graph', comment='HMM graph')

for i in range(matrix.shape[0]):
    dot.node(str(i), str(i))

for i in range(matrix.shape[0]):
    for j in range(matrix.shape[1]):
        prob = matrix[i, j]
        if prob > min_prob_to_plot_an_edge:
            formated_prob = float_format.format(prob)
            dot.edge(str(i), str(j), label = float_format.format(prob))
dot

# Анимация аватара
Ключевое в данном блокноте именно возможность обратного хода, то есть генерации поведения после обучения. HMM генерирует временной ряд FACS Action Units, по которым анимируется аватар. Для простоты интеграции с jupiter используется примитивная картинка из библиотеки `py-feat`, однако существуют готовые решения, которые позволяют по Action Units 3d модели, о них ниже.

## Анимация по FACS Action Units
### FACS Avatar
Докер контейнер, позволяющий анимировать при помощи FACS action units модель в Blender, Unity или MakeHuman. [GitHub](https://github.com/NumesSanguis/FACSvatar), лицензия LGPL. Был собран на базе нескольких других готовых проектов одним человеком, сейчас не поддерживается.

In [ ]:
from IPython.display import Image
Image(filename='../why_FACS_control_trans.png')

### OpenFace FACS Unity Facial Animator
Докер контейнер, который позволяет при помощи Action Units анимировать аватар на Unity.
[Github](https://github.com/alexismorin/OpenFace-FACS-Unity-Facial-Animator), лицензия LGPL. Входит в состав FACS Avatar.

### OpenFacs
Код на C++, которые локально рендерит аватар и позволят динамически менять его выражение лица посылая новые Action Units по UDP. Проект единожды написанный для статьи и с тех пор заброшен, но запускается и работает. [GitHub](https://github.com/phuselab/openFACS), лицензия MIT.

## Анимация в этом блокноте
Признаки выделялись только в каждом 25-ом кадре, то есть каждую секунду. Из-за этого движение глаз и рта хаотичны как на модели "оригинале", так и на аватаре.

In [ ]:
from celluloid import Camera
from IPython.display import HTML
from feat.plotting import plot_face

def animate_au(fig, ax, aus, *args, **kwargs,):
    camera = Camera(fig)
    
    for au in tqdm.tqdm(aus):
        # ax.clear()
        ax = plot_face(
            model=None,
            ax=ax,
            au=au,
            *args,
            **kwargs,
        )
        camera.snap()
    ani = camera.animate()
    return ani

In [ ]:
animation_fps = 10
animation_dpi = 300
animation_filename = "input.gif"

fig, ax = plt.subplots(figsize=(4,5))
ani = animate_au(fig, ax, X / float(raw_data_multiplier))
print("Person's face")
HTML(ani.to_jshtml())

In [ ]:
fig, ax = plt.subplots(figsize=(4,5))
ani = animate_au(fig, ax, model.lambdas_[states] / float(raw_data_multiplier))
print("Avatar's face")
HTML(ani.to_jshtml())

# Эмоции как исходные данные для HMM
В примере выше эмоции вообще никак не учитываются при моделировании. HMM принимает на вход только наблюдаемые Action Units, эмоций для нее не существует.

Может оказаться, что лучше все таки концепцию эмоций использовать. Тогда пути данных в прямом и обратном направлении будут такими:
```
Image -> Land Marks -> FACS Action Units -> Emotions -> HMM
```
```
HMM -> Emotions -> FACS Action Units -> Animator -> Image
```
## Распознавание эмоций
Эмоции по FACS Action Units можно распознавать при помощи ML моделей, так и делает библиотека `py-feat`. Проблема в том, такое преобразование получается необратимым. По Action Units эмоции восстановить можно, а как из модели достать обратное преобразование не ясно.

Будем использовать другой, более простой и не связанный с ML
подход. Назовем эмоцией подпространство в пространстве action
units. Выражение лица описывается вектором в Action Units пространстве. Степень выражения соответствия выражения лица эмоции равна углу между подпространством эмоции и вектором выражения лица.

Некоторые эмоции могут иметь несколько способов выражения. Так удивляясь человек может приоткрыть рот, а может и нет. Правильнее сказать, что что эмоция -- класс эквивалентности подпространств в пространстве Action Units. Но будем для простоты считать, удивление с открытым ртом и закрытым -- разные эмоции.

TODO: Refactoring, Выделить связанные с эмоциями функции и структуры данных в класс.

In [ ]:
def auName2auNum(name : str):
    return int(name[-2:])
def auNum2auName(number : int):
    return f"AU{int(number):02d}"
def test_auName2auNum_auNum2auName():
    for name in list(data.aus.columns):
        number = auName2auNum(name)
        name_test = auNum2auName(number)
        if (name_test != name_test):
            print(f"Failed {name} -> {number} -> {name_test}")
            assert(False)
test_auName2auNum_auNum2auName()

Данные об эмоциях-подпространствах возьмем из книги Paul Ekman, Wallace V. Friesen, Joseph C. Hager, "Facial Action Coding System Investigator’s Guide". В книге вводится интуитивно понятный DSL для их описания в виде формальных сумм. Звездочка (*) около номера Action Unit означает, что этот Action Unit может быть выражен в любой степени.

In [ ]:
# table from Facial Action Coding System Investigator’s Guide, by Paul Ekman, Wallace V. Friesen, Joseph C. Hager
emotions = {
    'anger' : {
        '4+5*+7+10*+22+23+25',
        '4+5*+7+23+25',
        '4+5*+7+10*+22+23+26',
        '4+5*+7+23+26',
        '4+5*+7+17+23',
        '4+5*+7+17+24',
        '4+5*+7+23',
        '4+5*+7+24',
    },
    'disgust' : {
        '9',
        '9+16+15',
        '9+16+26',
        '9+17',
        '10*',
        '10*+16+26',
        '10*+16+25',
        '10+17',
    },
    'fear' : {
        '1+2+4+5*+20*+25',
        '1+2+4+5*+25',
        '1+2+4+5*+20*+26',
        '1+2+4+5*+26',
        '1+2+4+5*+20*+27',
        '1+2+4+5*+27',
    }, 
    'happiness' : {
        '6+12*',
        '12',
    },
    'sadness' : {
        '1+4+11+15',
        '1+4+15*',
        '6+15*',
    },
    'surprise' : {
        '1+2+5+26',
        '1+2+5+27',
    },
    #'neutral' : {}
}

Подпространство эмоции будем описывать при помощи ортогонально проецирующей матрицы. `parseAdditiveCombination` конвертируют строку формальной суммы в матрицу-проектор на подпространство.

In [ ]:
def parseAdditiveCombination(combination : str, aus_space : list):
    au_strs    = combination.split('+')
    N          = len(aus_space)
    vector_sum = np.zeros(N)
    direct_sum = []
    for au_str in au_strs:
        try:
            add_to_direct_summ = (au_str[-1] == '*')
            if add_to_direct_summ:
                au_str = au_str[:-1] # remove asterisk
            if aus_space.count(auNum2auName(au_str)) <= 0:
                continue
        except:
            raise ValueError(f"Invelid term \"{au_str}\" in additive combination \"{combination}\"")

        vector = np.zeros(N)
        vector[aus_space.index(auNum2auName(au_str))] = 1
        
        if add_to_direct_summ:
            direct_sum.append(vector)
        else:
            vector_sum = vector_sum + vector
    vector_sum_norm = np.linalg.norm(vector_sum)
    if vector_sum_norm != 0:
        vector_sum = vector_sum / vector_sum_norm
        direct_sum.append(vector_sum)
    # TODO check that row of matrix to return is orthogonal
    return None if len(direct_sum) == 0 else np.stack(direct_sum)

def test_parseAdditiveCombination():
    N = 3
    au_labes = list(map(auNum2auName, np.arange(N) + 1)) # ['AU01', 'AU2', ...]
    ones = np.repeat(1, N)
    assert (parseAdditiveCombination('1*+2*+3*', au_labes) == np.identity(N)).all()
    assert (parseAdditiveCombination('1+2+3', au_labes) == ones / np.linalg.norm(ones)).all()
    assert parseAdditiveCombination('42', au_labes) is None

test_parseAdditiveCombination()

Как уже было сказано выше, степень варажения соответствия выражения лица эмоции равнауглу между подпространством эмоции и ветором выражения лица.

In [ ]:
def emotion_value(emotion_projector: np.ndarray, face_expression : np.ndarray):
    return scipy.linalg.subspace_angles(emotion_projector.T, face_expression.T)

In [ ]:
emotion_spaces = {}
for emotion_name, sum_strs in emotions.items():
    subspaces = []
    for sum_str in sum_strs:
        subspace = parseAdditiveCombination(sum_str, list(data.aus.columns))
        if len(subspaces) == 0:
            subspaces.append(subspace)
            continue
        # check if subspace have already been added
        angles = map(emotion_value, itertools.repeat(subspace), subspaces)
        there_is_a_zero_angle = sum(list(map(lambda angles_arr: (angles_arr == 0).any(), angles)))
        if there_is_a_zero_angle:
            subspaces.append(subspace)
    emotion_spaces[emotion_name] = subspaces

# display(emotion_spaces)